# Modello euristico

## Import pacchetti

In [1]:
# ------------------------------------------------------------
# 0) IMPORT
# ------------------------------------------------------------
from tabnanny import verbose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle


# Notebook esterno con tutte le funzioni di routing
import import_ipynb
import performance_calc as pc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650547 entries, 0 to 650546
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   location_id     650547 non-null  int64  
 1   lat             650547 non-null  float64
 2   lon             650547 non-null  float64
 3   quantity        650547 non-null  int64  
 4   delivery_date   650547 non-null  object 
 5   window_start_0  639992 non-null  object 
 6   window_end_0    639992 non-null  object 
 7   is_event        650547 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 39.7+ MB
❌ Valori non convertiti in 'delivery_date':
[]

❌ Valori non convertibili in booleani in 'is_event':
[]


<string>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ True False False ...  True  True  True]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


❌ Valori non convertiti in window_start_0:
[None]

❌ Valori non convertiti in window_end_0:
[None]
<class 'pandas.core.frame.DataFrame'>
Index: 63733 entries, 0 to 650546
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   location_id     63733 non-null  int64         
 1   lat             63733 non-null  float64       
 2   lon             63733 non-null  float64       
 3   quantity        63733 non-null  int64         
 4   delivery_date   63733 non-null  datetime64[ns]
 5   window_start_0  63590 non-null  object        
 6   window_end_0    63590 non-null  object        
 7   is_event        63733 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 4.4+ MB
Punti di consegna unici trovati: 3766
Dopo l'eliminazione di punti troppo distanti, sono rimasti 3764 punti di consegna
delivery points esempio:
   location_id        lat        lon
0         2884

## Implementazione modello euristico

## Last try - Problemi memoria

In [2]:

def clustering_euristico_multicluster(full_time_mat_min: np.ndarray,
                                      index_to_location_id: list[int],
                                      delivery_points: pd.DataFrame,
                                      max_time: int = 500,
                                      unloading_time: int = 10):
    """
    Algoritmo Cluster‐First, Route‐Second:
    • seed = punto più distante dal deposito
    • greedy: aggiunge il punto che allunga meno il tour
    • stop quando supererebbe max_time
    """
    n_points = full_time_mat_min.shape[0]
    unassigned = set(range(1, n_points))          # deposito è indice 0
    clusters = []


    while unassigned:
        # --- nuovo cluster
        first_idx = max(unassigned, key=lambda i: full_time_mat_min[0, i])
        cluster = [first_idx]
        unassigned.remove(first_idx)
        cluster_time = full_time_mat_min[0, first_idx] * 2 + unloading_time


        # --- inserimento greedy
        while unassigned:
            # tempo minimo dal cluster al candidato
            cand, min_cost = min(
                ((c, min(full_time_mat_min[p, c] for p in cluster)) for c in unassigned),
                key=lambda x: x[1]
            )
            if cluster_time + min_cost + unloading_time > max_time:
                break
            cluster.append(cand)
            unassigned.remove(cand)
            cluster_time += min_cost + unloading_time


        clusters.append([index_to_location_id[i] for i in cluster])


    return clusters



def plot_clusters(clusters, delivery_points, title):
    """Visual plot rapido"""
    cmap = plt.cm.get_cmap("tab20", len(clusters))
    plt.figure(figsize=(10, 7))
    for i, cl in enumerate(clusters):
        sub = delivery_points[delivery_points["location_id"].isin(cl)]
        plt.scatter(sub["lon"], sub["lat"], c=[cmap(i)],
                    label=f"C{i+1}", alpha=.7, s=20, edgecolors="k")
    plt.title(title)
    plt.xlabel("lon"); plt.ylabel("lat")
    plt.legend(ncol=2, fontsize="small")
    plt.show()


def deduplicate_clusters(clusters, already_assigned_points):
    """
    Rimuove punti già assegnati dai cluster e restituisce solo cluster validi
    
    Args:
        clusters: lista di cluster (liste di location_id)
        already_assigned_points: set di location_id già assegnati
        
    Returns:
        result: lista di cluster puliti
        assigned: set aggiornato di punti assegnati
    """
    assigned = set(already_assigned_points)
    result = []
    
    for cluster in clusters:
        clean_cluster = [loc_id for loc_id in cluster if loc_id not in assigned]
        if clean_cluster:  # Solo se il cluster ha almeno un punto
            result.append(clean_cluster)
            assigned.update(clean_cluster)
    
    return result, assigned


def get_last_iteration_rejected_clusters(all_results, already_assigned_points, verbose=True):
    """
    Recupera i cluster RIFIUTATI dell'ULTIMA iterazione, filtrando i punti già assegnati
    
    Args:
        all_results: dizionario con i risultati di tutte le iterazioni
        already_assigned_points: set di location_id già assegnati
        verbose: stampa messaggi di debug
        
    Returns:
        lista di cluster filtrati
    """
    if not all_results:
        return []
    
    # Prendi l'ultima iterazione
    last_iteration = max(all_results.keys())
    last_results = all_results[last_iteration]
    
    # Recupera i cluster rifiutati
    rejected_cluster_indices = last_results['rejected_indices']
    all_clusters = last_results['clusters']
    rejected_clusters = [all_clusters[i] for i in rejected_cluster_indices]
    
    if verbose:
        print(f"   🔍 Recuperando cluster rifiutati dall'ultima iterazione ({last_iteration})")
        print(f"   📋 Cluster rifiutati disponibili: {len(rejected_clusters)}")
    
    # Filtra punti già assegnati
    filtered_clusters, _ = deduplicate_clusters(rejected_clusters, already_assigned_points)
    
    if verbose:
        total_points = sum(len(cl) for cl in filtered_clusters)
        print(f"   ✅ Cluster recuperati dopo filtro: {len(filtered_clusters)} ({total_points} punti)")
    
    return filtered_clusters


def count_clusters_with_excessive_overtime(performance_df, threshold=1):
    """
    Conta quanti cluster hanno più di 'threshold' giorni di overtime
    
    Args:
        performance_df: DataFrame con le performance dei cluster
        threshold: soglia di overtime (default 1)
        
    Returns:
        numero di cluster con overtime > threshold
    """
    count = 0
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        total_overtime = cluster_data['n_overtime_days'].sum()
        if total_overtime > threshold:
            count += 1
    return count



def evaluate_and_accept_clusters(clusters, performance_df, verbose=True):
    """
    Valuta i cluster e decide quali accettare basandosi sui criteri:
    1) Media giornaliera tra 390-480 minuti
    2) Overtime accettabile (massimo 1 giorno per tipo di giorno della settimana)
    
    Returns:
        accepted_clusters: list di cluster (liste di location_id) accettati
        rejected_indices: list di indici dei cluster rifiutati
    """
    accepted_clusters = []
    rejected_indices = []
    
    # Raggruppa performance per cluster
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        cluster_index = int(cluster_name.split()[-1]) - 1  # "Cluster 1" -> index 0
        cluster = clusters[cluster_index]
        
        # CRITERIO 1: Media giornaliera tra 390-480 minuti
        mean_minutes_range = cluster_data['mean_minutes'].between(390, 480).any()
        
        # CRITERIO 2: Overtime accettabile
        overtime_acceptable = check_overtime_acceptable(cluster_data)
        
        if mean_minutes_range or overtime_acceptable:
            accepted_clusters.append(cluster)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                reason = "mean_range" if mean_minutes_range else "overtime_ok"
                print(f"      ✅ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"motivo={reason}")
        else:
            rejected_indices.append(cluster_index)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                print(f"      ❌ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"RIFIUTATO")
    
    return accepted_clusters, rejected_indices



def check_overtime_acceptable(cluster_data):
    """
    Controlla se l'overtime è accettabile:
    massimo 1 giorno di overtime per ogni giorno della settimana
    
    Args:
        cluster_data: DataFrame con performance di un singolo cluster
    
    Returns:
        bool: True se overtime accettabile, False altrimenti
    """
    # Controlla per ogni giorno della settimana
    for _, row in cluster_data.iterrows():
        if row['n_overtime_days'] > 1:  # Più di 1 giorno di overtime per questo weekday -> non accettato
            return False
        elif row['n_overtime_days'] == 1:
            return True  # Ok, 1 giorno di overtime è accettabile
    
    return False






def assemble_final_performance_from_cache(all_results, final_complete_clusters, verbose=True):
    """
    Assembla un DataFrame finale dalle performance già calcolate nelle iterazioni.
    Molto più veloce del ricalcolo completo.
    """
    if verbose:
        print("📋 Assemblando performance finali da cache esistenti...")
    
    final_performance_pieces = []
    cluster_counter = 1
    
    for cluster in final_complete_clusters:
        cluster_found = False
        
        # Cerca questo cluster nelle iterazioni passate
        for iteration_num, results in all_results.items():
            for i, iter_cluster in enumerate(results['clusters']):
                if set(iter_cluster) == set(cluster):  # Stesso cluster
                    # Prendi le performance già calcolate
                    iter_perf_df = results['performance_df']
                    cluster_name_old = f"Cluster {i+1}"
                    cluster_data = iter_perf_df[iter_perf_df['cluster'] == cluster_name_old].copy()
                    
                    # Rinomina con il nuovo indice
                    cluster_data['cluster'] = f"Cluster {cluster_counter}"
                    final_performance_pieces.append(cluster_data)
                    cluster_counter += 1
                    cluster_found = True
                    break
            
            if cluster_found:
                break
        
        if not cluster_found and verbose:
            print(f"   ⚠️ Cluster non trovato in cache - dovrà essere ricalcolato")
    
    if final_performance_pieces:
        final_df = pd.concat(final_performance_pieces, ignore_index=True)
        if verbose:
            print(f"   ✅ Assemblate performance per {len(final_performance_pieces)} cluster da cache")
        return final_df
    else:
        if verbose:
            print("   ⚠️ Nessuna performance in cache - calcolo completo necessario")
        return None



def heuristic_iterative_clustering_complete(delivery_points: pd.DataFrame,
                                           depot_location: tuple[float, float],
                                           initial_threshold: int = 800,
                                           increment_threshold: int = 120,
                                           unload_min: int = 10,
                                           max_iterations: int = 10,
                                           verbose: bool = True,
                                           enable_diagnostics: bool = False):
    """
    Algoritmo iterativo che restituisce TUTTI i punti assegnati a cluster.
    MODIFICHE:
    - Ogni punto assegnato ad un solo cluster (no duplicati)
    - Fallback all'ultima iterazione per punti non assegnati
    - Refinement automatico se >5 cluster con overtime >1
    
    Returns:
        all_results: dict con tutti i risultati intermedi
        final_complete_clusters: list di TUTTI i cluster (accettati + migliori fallback)
        final_performance: DataFrame con performance di tutti i cluster finali
        refinement_applied: bool indicante se è stato applicato refinement
    """
    
    if verbose:
        print("🔄 INIZIO clustering iterativo COMPLETO con soglia crescente")
        print(f"   • Soglia iniziale: {initial_threshold} min")
        print(f"   • Incremento: +{increment_threshold} min per iterazione")
    
    all_results = {}
    remaining_points = delivery_points.copy()
    original_points = set(delivery_points['location_id'])
    already_assigned_points = set()  # NUOVO: Traccia globale dei punti già assegnati
    current_threshold = initial_threshold
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n🔄 Iterazione {iteration + 1}: soglia = {current_threshold} min")
            print(f"   • Punti rimanenti: {len(remaining_points)}")
        
        if remaining_points.empty:
            if verbose:
                print("   ✅ Nessun punto rimanente - STOP")
            break
        
        # ================== CLUSTERING ==================
        dm, tm, idx2loc, _ = pc.distance_matrix_creation('', remaining_points)
        clusters = clustering_euristico_multicluster(
            tm, idx2loc, remaining_points,
            max_time=current_threshold,
            unloading_time=unload_min
        )
        
        if not clusters:
            if verbose:
                print("   ⚠️ Nessun cluster generato - STOP")
            break
        
        # ================== PERFORMANCE ==================
        performance_df = pc.calc_clusters_stats_AS(
            clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        # ================== VALUTAZIONE E ACCETTAZIONE ==================
        accepted_clusters, rejected_cluster_indices = evaluate_and_accept_clusters(
            clusters, performance_df, verbose=verbose
        )
        
        # Salva risultati dell'iterazione (con rejected_indices invece di rejected_clusters)
        all_results[iteration + 1] = {
            'threshold': current_threshold,
            'clusters': clusters,
            'performance_df': performance_df,
            'accepted_clusters': accepted_clusters,
            'rejected_indices': rejected_cluster_indices
        }
        
        if verbose:
            print(f"   ✅ Cluster accettati: {len(accepted_clusters)}/{len(clusters)}")
        
        # ================== RIMOZIONE PUNTI ACCETTATI (NO DUPLICATI) ==================
        accepted_location_ids = set()
        for cluster in accepted_clusters:
            accepted_location_ids.update(cluster)
        
        # NUOVO: Filtra solo gli ID che esistono effettivamente nel DataFrame
        existing_ids = set(remaining_points['location_id'])
        valid_accepted_ids = accepted_location_ids.intersection(existing_ids)
        missing_ids = accepted_location_ids - existing_ids
        
        if verbose and missing_ids:
            print(f'   ⚠️ ID nei cluster ma non nel DataFrame: {len(missing_ids)} ID')
            print(f'   🔧 ID validi da rimuovere: {len(valid_accepted_ids)} ID')
        
        # NUOVO: Aggiorna il set globale dei punti assegnati
        already_assigned_points.update(valid_accepted_ids)

        # PER LA DIAGNOSTICA (opzionale):
        if enable_diagnostics and verbose:
            remaining_points_after_removal = remaining_points[
                ~remaining_points["location_id"].isin(valid_accepted_ids)
            ].copy()
            diagnose_point_removal(remaining_points, accepted_clusters, 
                remaining_points_after_removal, iteration + 1)
        
        # USA VALID_ACCEPTED_IDS invece di ACCEPTED_LOCATION_IDS
        remaining_points = remaining_points[
            ~remaining_points['location_id'].isin(valid_accepted_ids)
        ].copy()
        
        # Se tutti i cluster sono stati accettati, abbiamo finito
        if len(accepted_clusters) == len(clusters):
            if verbose:
                print("   🎯 Tutti i cluster sono stati accettati - STOP")
            break
        
        # Incrementa soglia per prossima iterazione
        current_threshold += increment_threshold
    
    # ================== ASSEMBLAGGIO FINALE ==================
    if verbose:
        print(f"\n🔧 Assemblaggio soluzione finale...")
    
    final_complete_clusters = []
    
    # 1. Aggiungi tutti i cluster accettati (già senza duplicati)
    for iteration_results in all_results.values():
        for cluster in iteration_results['accepted_clusters']:
            final_complete_clusters.append(cluster)
    
    # 2. NUOVO: Per i punti rimanenti, usa i cluster RIFIUTATI dell'ULTIMA ITERAZIONE
    unassigned_points = original_points - already_assigned_points
    
    if unassigned_points:
        if verbose:
            print(f"   📋 Punti non assegnati: {len(unassigned_points)}")
            print("   🔍 Recuperando cluster rifiutati dall'ultima iterazione...")
        
        last_iteration_clusters = get_last_iteration_rejected_clusters(
            all_results, already_assigned_points, verbose
        )
        
        final_complete_clusters.extend(last_iteration_clusters)
        for cluster in last_iteration_clusters:
            already_assigned_points.update(cluster)
    
    # ================== PERFORMANCE FINALI ==================
    if verbose:
        print(f"\n📊 Calcolando performance finali per {len(final_complete_clusters)} cluster...")
    
    # Calcola performance (senza cache per ora, per evitare problemi)
    final_performance = pc.calc_clusters_stats_AS(
        final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
    )
    
    if verbose:
        total_points = sum(len(cluster) for cluster in final_complete_clusters)
        print(f"\n🏁 COMPLETATO:")
        print(f"   • Iterazioni eseguite: {len(all_results)}")
        print(f"   • Cluster finali totali: {len(final_complete_clusters)}")
        print(f"   • Punti totali assegnati: {total_points}")
        print(f"   • Copertura: {total_points}/{len(original_points)} punti")
        
        missing = len(original_points) - total_points
        if missing > 0:
            print(f"   ⚠️ Punti mancanti: {missing}")
    
    # ================== CHECK OVERTIME ECCESSIVO E REFINEMENT ==================
    excessive_overtime_count = count_clusters_with_excessive_overtime(final_performance, threshold=1)
    
    if verbose:
        print(f"\n📊 Cluster con più di 1 giorno overtime: {excessive_overtime_count}")
    
    # NUOVO: Se ci sono più di 5 cluster con overtime eccessivo, applica refinement
    refinement_needed = excessive_overtime_count > 5
    
    if refinement_needed:
        if verbose:
            print(f"\n🔄 REFINEMENT NECESSARIO: {excessive_overtime_count} cluster con overtime > 1")
            print("   🚀 Avvio 5 cicli aggiuntivi con soglia 1550 min (+60/iterazione)...")
        
        # Identifica i punti nei cluster problematici
        problematic_points = set()
        problematic_cluster_names = []
        for cluster_name, cluster_data in final_performance.groupby('cluster'):
            total_overtime = cluster_data['n_overtime_days'].sum()
            if total_overtime > 1:
                problematic_cluster_names.append(cluster_name)
                cluster_index = int(cluster_name.split()[-1]) - 1
                problematic_cluster = final_complete_clusters[cluster_index]
                problematic_points.update(problematic_cluster)
        
        # Crea DataFrame con i punti problematici
        refinement_points = delivery_points[
            delivery_points['location_id'].isin(problematic_points)
        ].copy()
        
        if verbose:
            print(f"   📋 Cluster problematici: {len(problematic_cluster_names)}")
            print(f"   📋 Punti da riclusterare: {len(refinement_points)}")
        
        # Applica refinement (ricorsivo)
        refinement_results, refinement_clusters, refinement_performance, _ = heuristic_iterative_clustering_complete(
            delivery_points=refinement_points,
            depot_location=depot_location,
            initial_threshold=1550,  # NUOVO: soglia più alta
            increment_threshold=60,   # NUOVO: incremento più piccolo
            unload_min=unload_min,
            max_iterations=5,        # NUOVO: max 5 iterazioni
            verbose=verbose,
            enable_diagnostics=False  # Disabilita diagnostica nel refinement
        )
        
        # Sostituisci i cluster problematici con quelli raffinati
        final_complete_clusters = [
            cl for cl in final_complete_clusters 
            if not any(p in problematic_points for p in cl)
        ]
        final_complete_clusters.extend(refinement_clusters)
        
        # Ricalcola performance finali
        final_performance = pc.calc_clusters_stats_AS(
            final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        if verbose:
            print(f"\n✅ REFINEMENT COMPLETATO")
            print(f"   • Nuovi cluster totali: {len(final_complete_clusters)}")
            excessive_after = count_clusters_with_excessive_overtime(final_performance, threshold=1)
            print(f"   • Cluster con overtime > 1: {excessive_after} (prima: {excessive_overtime_count})")
        
        return all_results, final_complete_clusters, final_performance, True
    
    return all_results, final_complete_clusters, final_performance, False





def plot_iteration_results(all_results, delivery_points, iteration_to_plot=None):
    """
    Plotta i risultati di una specifica iterazione o dell'ultima
    """
    if iteration_to_plot is None:
        iteration_to_plot = max(all_results.keys())
    
    if iteration_to_plot not in all_results:
        print(f"Iterazione {iteration_to_plot} non trovata")
        return
    
    results = all_results[iteration_to_plot]
    clusters = results['clusters']
    threshold = results['threshold']
    accepted = results['accepted_clusters']
    
    plot_clusters(clusters, delivery_points, 
                  f"Iterazione {iteration_to_plot} - Soglia {threshold} min "
                  f"({len(accepted)} accettati/{len(clusters)} totali)")



# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# SOLO PER DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


def diagnose_point_removal(remaining_points_before, accepted_clusters, remaining_points_after, iteration):
    """Diagnostica dettagliata per capire la discrepanza"""
    
    print(f"\n🔍 DIAGNOSTICA ITERAZIONE {iteration}:")
    
    # 1. Conta punti prima
    points_before = len(remaining_points_before)
    print(f"   📊 Punti prima: {points_before}")
    
    # 2. Conta punti negli cluster accettati
    accepted_location_ids = set()
    total_accepted_points = 0
    for i, cluster in enumerate(accepted_clusters):
        cluster_size = len(cluster)
        total_accepted_points += cluster_size
        accepted_location_ids.update(cluster)
        print(f"   ✅ Cluster {i+1}: {cluster_size} punti")
    
    print(f"   📊 Totale punti accettati: {total_accepted_points}")
    print(f"   📊 Location_ID unici accettati: {len(accepted_location_ids)}")
    
    # 3. Verifica che tutti i location_id accettati esistano in remaining_points_before
    existing_ids = set(remaining_points_before['location_id'])
    missing_ids = accepted_location_ids - existing_ids
    if missing_ids:
        print(f"   ⚠️ PROBLEMA: {len(missing_ids)} ID accettati non esistono in remaining_points!")
        print(f"      ID mancanti: {list(missing_ids)[:10]}...")
    
    # 4. Simula la rimozione manualmente
    should_remain = remaining_points_before[
        ~remaining_points_before['location_id'].isin(accepted_location_ids)
    ]
    expected_remaining = len(should_remain)
    
    # 5. Conta punti dopo
    points_after = len(remaining_points_after)
    print(f"   📊 Punti dopo rimozione: {points_after}")
    print(f"   📊 Punti attesi dopo rimozione: {expected_remaining}")
    
    # 6. Calcola discrepanze
    actual_removed = points_before - points_after
    expected_removed = len(accepted_location_ids)
    discrepancy = total_accepted_points - actual_removed
    
    print(f"   📊 Rimossi effettivamente: {actual_removed}")
    print(f"   📊 Dovevano essere rimossi: {expected_removed}")
    print(f"   📊 DISCREPANZA: {discrepancy}")
    
    if discrepancy != 0:
        print(f"   ❌ DISCREPANZA RILEVATA!")
        
        # Analisi più profonda
        if points_after != expected_remaining:
            print(f"   🔍 Il DataFrame finale non corrisponde alla simulazione")
            print(f"   🔍 Differenza: {points_after - expected_remaining}")
    
    print(f"   ✅ Diagnostica completata\n")
    
    return {
        'points_before': points_before,
        'total_accepted_points': total_accepted_points,
        'actual_removed': actual_removed,
        'discrepancy': discrepancy
    }

# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# FINE DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


### Run

In [3]:
import time
start = time.time()

all_results, final_clusters, final_performance, refinement_applied = heuristic_iterative_clustering_complete(
    delivery_points=pc.delivery_points_AS,
    depot_location=pc.depot_location,
    initial_threshold=800,
    increment_threshold=100,
    unload_min=10,
    max_iterations=15,
    verbose=True,
    enable_diagnostics=True  # Metti False per disabilitare la diagnostica
)


end = time.time()
print(f"Tempo di esecuzione algoritmo: {(end - start)/60:.2f} min")


# Stampa risultato refinement
if refinement_applied:
    print("\n🎯 REFINEMENT APPLICATO con successo!")
else:
    print("\n✅ NESSUN REFINEMENT necessario - soluzione ottimale trovata")

🔄 INIZIO clustering iterativo COMPLETO con soglia crescente
   • Soglia iniziale: 800 min
   • Incremento: +100 min per iterazione

🔄 Iterazione 1: soglia = 800 min
   • Punti rimanenti: 2972
      ✅ Cluster 1: 41 punti, max_mean=461.4min, overtime=2, motivo=mean_range
      ❌ Cluster 10: 52 punti, max_mean=268.9min, overtime=0, RIFIUTATO
      ✅ Cluster 11: 50 punti, max_mean=313.1min, overtime=1, motivo=overtime_ok
      ❌ Cluster 12: 57 punti, max_mean=320.9min, overtime=0, RIFIUTATO
      ❌ Cluster 13: 57 punti, max_mean=270.7min, overtime=0, RIFIUTATO
      ❌ Cluster 14: 50 punti, max_mean=213.5min, overtime=0, RIFIUTATO
      ❌ Cluster 15: 54 punti, max_mean=260.0min, overtime=0, RIFIUTATO
      ❌ Cluster 16: 55 punti, max_mean=231.1min, overtime=0, RIFIUTATO
      ❌ Cluster 17: 58 punti, max_mean=183.5min, overtime=0, RIFIUTATO
      ❌ Cluster 18: 54 punti, max_mean=256.6min, overtime=0, RIFIUTATO
      ❌ Cluster 19: 54 punti, max_mean=289.5min, overtime=0, RIFIUTATO
      ❌ Clu

In [ ]:

final_performance.to_csv('clustering_methods_performances/euristic_complete_v2_AS_5.csv')

with open('cluster_dicts/cluster_dict_euristic_complete_v2_AS_5.pkl', 'wb') as f:
    pickle.dump(final_clusters, f)

# 2 ON

In [5]:

def clustering_euristico_multicluster(full_time_mat_min: np.ndarray,
                                      index_to_location_id: list[int],
                                      delivery_points: pd.DataFrame,
                                      max_time: int = 500,
                                      unloading_time: int = 10):
    """
    Algoritmo Cluster‐First, Route‐Second:
    • seed = punto più distante dal deposito
    • greedy: aggiunge il punto che allunga meno il tour
    • stop quando supererebbe max_time
    """
    n_points = full_time_mat_min.shape[0]
    unassigned = set(range(1, n_points))          # deposito è indice 0
    clusters = []


    while unassigned:
        # --- nuovo cluster
        first_idx = max(unassigned, key=lambda i: full_time_mat_min[0, i])
        cluster = [first_idx]
        unassigned.remove(first_idx)
        cluster_time = full_time_mat_min[0, first_idx] * 2 + unloading_time


        # --- inserimento greedy
        while unassigned:
            # tempo minimo dal cluster al candidato
            cand, min_cost = min(
                ((c, min(full_time_mat_min[p, c] for p in cluster)) for c in unassigned),
                key=lambda x: x[1]
            )
            if cluster_time + min_cost + unloading_time > max_time:
                break
            cluster.append(cand)
            unassigned.remove(cand)
            cluster_time += min_cost + unloading_time


        clusters.append([index_to_location_id[i] for i in cluster])


    return clusters



def plot_clusters(clusters, delivery_points, title):
    """Visual plot rapido"""
    cmap = plt.cm.get_cmap("tab20", len(clusters))
    plt.figure(figsize=(10, 7))
    for i, cl in enumerate(clusters):
        sub = delivery_points[delivery_points["location_id"].isin(cl)]
        plt.scatter(sub["lon"], sub["lat"], c=[cmap(i)],
                    label=f"C{i+1}", alpha=.7, s=20, edgecolors="k")
    plt.title(title)
    plt.xlabel("lon"); plt.ylabel("lat")
    plt.legend(ncol=2, fontsize="small")
    plt.show()


def deduplicate_clusters(clusters, already_assigned_points):
    """
    Rimuove punti già assegnati dai cluster e restituisce solo cluster validi
    
    Args:
        clusters: lista di cluster (liste di location_id)
        already_assigned_points: set di location_id già assegnati
        
    Returns:
        result: lista di cluster puliti
        assigned: set aggiornato di punti assegnati
    """
    assigned = set(already_assigned_points)
    result = []
    
    for cluster in clusters:
        clean_cluster = [loc_id for loc_id in cluster if loc_id not in assigned]
        if clean_cluster:  # Solo se il cluster ha almeno un punto
            result.append(clean_cluster)
            assigned.update(clean_cluster)
    
    return result, assigned


def get_last_iteration_rejected_clusters(all_results, already_assigned_points, verbose=True):
    """
    Recupera i cluster RIFIUTATI dell'ULTIMA iterazione, filtrando i punti già assegnati
    
    Args:
        all_results: dizionario con i risultati di tutte le iterazioni
        already_assigned_points: set di location_id già assegnati
        verbose: stampa messaggi di debug
        
    Returns:
        lista di cluster filtrati
    """
    if not all_results:
        return []
    
    # Prendi l'ultima iterazione
    last_iteration = max(all_results.keys())
    last_results = all_results[last_iteration]
    
    # Recupera i cluster rifiutati
    rejected_cluster_indices = last_results['rejected_indices']
    all_clusters = last_results['clusters']
    rejected_clusters = [all_clusters[i] for i in rejected_cluster_indices]
    
    if verbose:
        print(f"   🔍 Recuperando cluster rifiutati dall'ultima iterazione ({last_iteration})")
        print(f"   📋 Cluster rifiutati disponibili: {len(rejected_clusters)}")
    
    # Filtra punti già assegnati
    filtered_clusters, _ = deduplicate_clusters(rejected_clusters, already_assigned_points)
    
    if verbose:
        total_points = sum(len(cl) for cl in filtered_clusters)
        print(f"   ✅ Cluster recuperati dopo filtro: {len(filtered_clusters)} ({total_points} punti)")
    
    return filtered_clusters


def count_clusters_with_excessive_overtime(performance_df, threshold=1):
    """
    Conta quanti cluster hanno più di 'threshold' giorni di overtime
    
    Args:
        performance_df: DataFrame con le performance dei cluster
        threshold: soglia di overtime (default 1)
        
    Returns:
        numero di cluster con overtime > threshold
    """
    count = 0
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        total_overtime = cluster_data['n_overtime_days'].sum()
        if total_overtime > threshold:
            count += 1
    return count



def evaluate_and_accept_clusters(clusters, performance_df, verbose=True):
    """
    Valuta i cluster e decide quali accettare basandosi sui criteri:
    1) Media giornaliera tra 390-480 minuti
    2) Overtime accettabile (massimo 1 giorno per tipo di giorno della settimana)
    
    Returns:
        accepted_clusters: list di cluster (liste di location_id) accettati
        rejected_indices: list di indici dei cluster rifiutati
    """
    accepted_clusters = []
    rejected_indices = []
    
    # Raggruppa performance per cluster
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        cluster_index = int(cluster_name.split()[-1]) - 1  # "Cluster 1" -> index 0
        cluster = clusters[cluster_index]
        
        # CRITERIO 1: Media giornaliera tra 390-480 minuti
        mean_minutes_range = cluster_data['mean_minutes'].between(390, 480).any()
        
        # CRITERIO 2: Overtime accettabile
        overtime_acceptable = check_overtime_acceptable(cluster_data)
        
        if mean_minutes_range or overtime_acceptable:
            accepted_clusters.append(cluster)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                reason = "mean_range" if mean_minutes_range else "overtime_ok"
                print(f"      ✅ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"motivo={reason}")
        else:
            rejected_indices.append(cluster_index)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                print(f"      ❌ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"RIFIUTATO")
    
    return accepted_clusters, rejected_indices



def check_overtime_acceptable(cluster_data):
    """
    Controlla se l'overtime è accettabile:
    massimo 1 giorno di overtime per ogni giorno della settimana
    
    Args:
        cluster_data: DataFrame con performance di un singolo cluster
    
    Returns:
        bool: True se overtime accettabile, False altrimenti
    """
    # Controlla per ogni giorno della settimana
    for _, row in cluster_data.iterrows():
        if row['n_overtime_days'] > 1:  # Più di 1 giorno di overtime per questo weekday -> non accettato
            return False
        elif row['n_overtime_days'] == 1:
            return True  # Ok, 1 giorno di overtime è accettabile
    
    return False






def assemble_final_performance_from_cache(all_results, final_complete_clusters, verbose=True):
    """
    Assembla un DataFrame finale dalle performance già calcolate nelle iterazioni.
    Molto più veloce del ricalcolo completo.
    """
    if verbose:
        print("📋 Assemblando performance finali da cache esistenti...")
    
    final_performance_pieces = []
    cluster_counter = 1
    
    for cluster in final_complete_clusters:
        cluster_found = False
        
        # Cerca questo cluster nelle iterazioni passate
        for iteration_num, results in all_results.items():
            for i, iter_cluster in enumerate(results['clusters']):
                if set(iter_cluster) == set(cluster):  # Stesso cluster
                    # Prendi le performance già calcolate
                    iter_perf_df = results['performance_df']
                    cluster_name_old = f"Cluster {i+1}"
                    cluster_data = iter_perf_df[iter_perf_df['cluster'] == cluster_name_old].copy()
                    
                    # Rinomina con il nuovo indice
                    cluster_data['cluster'] = f"Cluster {cluster_counter}"
                    final_performance_pieces.append(cluster_data)
                    cluster_counter += 1
                    cluster_found = True
                    break
            
            if cluster_found:
                break
        
        if not cluster_found and verbose:
            print(f"   ⚠️ Cluster non trovato in cache - dovrà essere ricalcolato")
    
    if final_performance_pieces:
        final_df = pd.concat(final_performance_pieces, ignore_index=True)
        if verbose:
            print(f"   ✅ Assemblate performance per {len(final_performance_pieces)} cluster da cache")
        return final_df
    else:
        if verbose:
            print("   ⚠️ Nessuna performance in cache - calcolo completo necessario")
        return None



def heuristic_iterative_clustering_complete(delivery_points: pd.DataFrame,
                                           depot_location: tuple[float, float],
                                           initial_threshold: int = 800,
                                           increment_threshold: int = 120,
                                           unload_min: int = 10,
                                           max_iterations: int = 10,
                                           verbose: bool = True,
                                           enable_diagnostics: bool = False):
    """
    Algoritmo iterativo che restituisce TUTTI i punti assegnati a cluster.
    MODIFICHE:
    - Ogni punto assegnato ad un solo cluster (no duplicati)
    - Fallback all'ultima iterazione per punti non assegnati
    - Refinement automatico se >5 cluster con overtime >1
    
    Returns:
        all_results: dict con tutti i risultati intermedi
        final_complete_clusters: list di TUTTI i cluster (accettati + migliori fallback)
        final_performance: DataFrame con performance di tutti i cluster finali
        refinement_applied: bool indicante se è stato applicato refinement
    """
    
    if verbose:
        print("🔄 INIZIO clustering iterativo COMPLETO con soglia crescente")
        print(f"   • Soglia iniziale: {initial_threshold} min")
        print(f"   • Incremento: +{increment_threshold} min per iterazione")
    
    all_results = {}
    remaining_points = delivery_points.copy()
    original_points = set(delivery_points['location_id'])
    already_assigned_points = set()  # NUOVO: Traccia globale dei punti già assegnati
    current_threshold = initial_threshold
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n🔄 Iterazione {iteration + 1}: soglia = {current_threshold} min")
            print(f"   • Punti rimanenti: {len(remaining_points)}")
        
        if remaining_points.empty:
            if verbose:
                print("   ✅ Nessun punto rimanente - STOP")
            break
        
        # ================== CLUSTERING ==================
        dm, tm, idx2loc, _ = pc.distance_matrix_creation('', remaining_points)
        clusters = clustering_euristico_multicluster(
            tm, idx2loc, remaining_points,
            max_time=current_threshold,
            unloading_time=unload_min
        )
        
        if not clusters:
            if verbose:
                print("   ⚠️ Nessun cluster generato - STOP")
            break
        
        # ================== PERFORMANCE ==================
        performance_df = pc.calc_clusters_stats_ON(
            clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        # ================== VALUTAZIONE E ACCETTAZIONE ==================
        accepted_clusters, rejected_cluster_indices = evaluate_and_accept_clusters(
            clusters, performance_df, verbose=verbose
        )
        
        # Salva risultati dell'iterazione (con rejected_indices invece di rejected_clusters)
        all_results[iteration + 1] = {
            'threshold': current_threshold,
            'clusters': clusters,
            'performance_df': performance_df,
            'accepted_clusters': accepted_clusters,
            'rejected_indices': rejected_cluster_indices
        }
        
        if verbose:
            print(f"   ✅ Cluster accettati: {len(accepted_clusters)}/{len(clusters)}")
        
        # ================== RIMOZIONE PUNTI ACCETTATI (NO DUPLICATI) ==================
        accepted_location_ids = set()
        for cluster in accepted_clusters:
            accepted_location_ids.update(cluster)
        
        # NUOVO: Filtra solo gli ID che esistono effettivamente nel DataFrame
        existing_ids = set(remaining_points['location_id'])
        valid_accepted_ids = accepted_location_ids.intersection(existing_ids)
        missing_ids = accepted_location_ids - existing_ids
        
        if verbose and missing_ids:
            print(f'   ⚠️ ID nei cluster ma non nel DataFrame: {len(missing_ids)} ID')
            print(f'   🔧 ID validi da rimuovere: {len(valid_accepted_ids)} ID')
        
        # NUOVO: Aggiorna il set globale dei punti assegnati
        already_assigned_points.update(valid_accepted_ids)

        # PER LA DIAGNOSTICA (opzionale):
        if enable_diagnostics and verbose:
            remaining_points_after_removal = remaining_points[
                ~remaining_points["location_id"].isin(valid_accepted_ids)
            ].copy()
            diagnose_point_removal(remaining_points, accepted_clusters, 
                remaining_points_after_removal, iteration + 1)
        
        # USA VALID_ACCEPTED_IDS invece di ACCEPTED_LOCATION_IDS
        remaining_points = remaining_points[
            ~remaining_points['location_id'].isin(valid_accepted_ids)
        ].copy()
        
        # Se tutti i cluster sono stati accettati, abbiamo finito
        if len(accepted_clusters) == len(clusters):
            if verbose:
                print("   🎯 Tutti i cluster sono stati accettati - STOP")
            break
        
        # Incrementa soglia per prossima iterazione
        current_threshold += increment_threshold
    
    # ================== ASSEMBLAGGIO FINALE ==================
    if verbose:
        print(f"\n🔧 Assemblaggio soluzione finale...")
    
    final_complete_clusters = []
    
    # 1. Aggiungi tutti i cluster accettati (già senza duplicati)
    for iteration_results in all_results.values():
        for cluster in iteration_results['accepted_clusters']:
            final_complete_clusters.append(cluster)
    
    # 2. NUOVO: Per i punti rimanenti, usa i cluster RIFIUTATI dell'ULTIMA ITERAZIONE
    unassigned_points = original_points - already_assigned_points
    
    if unassigned_points:
        if verbose:
            print(f"   📋 Punti non assegnati: {len(unassigned_points)}")
            print("   🔍 Recuperando cluster rifiutati dall'ultima iterazione...")
        
        last_iteration_clusters = get_last_iteration_rejected_clusters(
            all_results, already_assigned_points, verbose
        )
        
        final_complete_clusters.extend(last_iteration_clusters)
        for cluster in last_iteration_clusters:
            already_assigned_points.update(cluster)
    
    # ================== PERFORMANCE FINALI ==================
    if verbose:
        print(f"\n📊 Calcolando performance finali per {len(final_complete_clusters)} cluster...")
    
    # Calcola performance (senza cache per ora, per evitare problemi)
    final_performance = pc.calc_clusters_stats_ON(
        final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
    )
    
    if verbose:
        total_points = sum(len(cluster) for cluster in final_complete_clusters)
        print(f"\n🏁 COMPLETATO:")
        print(f"   • Iterazioni eseguite: {len(all_results)}")
        print(f"   • Cluster finali totali: {len(final_complete_clusters)}")
        print(f"   • Punti totali assegnati: {total_points}")
        print(f"   • Copertura: {total_points}/{len(original_points)} punti")
        
        missing = len(original_points) - total_points
        if missing > 0:
            print(f"   ⚠️ Punti mancanti: {missing}")
    
    # ================== CHECK OVERTIME ECCESSIVO E REFINEMENT ==================
    excessive_overtime_count = count_clusters_with_excessive_overtime(final_performance, threshold=1)
    
    if verbose:
        print(f"\n📊 Cluster con più di 1 giorno overtime: {excessive_overtime_count}")
    
    # NUOVO: Se ci sono più di 5 cluster con overtime eccessivo, applica refinement
    refinement_needed = excessive_overtime_count > 5
    
    if refinement_needed:
        if verbose:
            print(f"\n🔄 REFINEMENT NECESSARIO: {excessive_overtime_count} cluster con overtime > 1")
            print("   🚀 Avvio 5 cicli aggiuntivi con soglia 1550 min (+60/iterazione)...")
        
        # Identifica i punti nei cluster problematici
        problematic_points = set()
        problematic_cluster_names = []
        for cluster_name, cluster_data in final_performance.groupby('cluster'):
            total_overtime = cluster_data['n_overtime_days'].sum()
            if total_overtime > 1:
                problematic_cluster_names.append(cluster_name)
                cluster_index = int(cluster_name.split()[-1]) - 1
                problematic_cluster = final_complete_clusters[cluster_index]
                problematic_points.update(problematic_cluster)
        
        # Crea DataFrame con i punti problematici
        refinement_points = delivery_points[
            delivery_points['location_id'].isin(problematic_points)
        ].copy()
        
        if verbose:
            print(f"   📋 Cluster problematici: {len(problematic_cluster_names)}")
            print(f"   📋 Punti da riclusterare: {len(refinement_points)}")
        
        # Applica refinement (ricorsivo)
        refinement_results, refinement_clusters, refinement_performance, _ = heuristic_iterative_clustering_complete(
            delivery_points=refinement_points,
            depot_location=depot_location,
            initial_threshold=1550,  # NUOVO: soglia più alta
            increment_threshold=60,   # NUOVO: incremento più piccolo
            unload_min=unload_min,
            max_iterations=5,        # NUOVO: max 5 iterazioni
            verbose=verbose,
            enable_diagnostics=False  # Disabilita diagnostica nel refinement
        )
        
        # Sostituisci i cluster problematici con quelli raffinati
        final_complete_clusters = [
            cl for cl in final_complete_clusters 
            if not any(p in problematic_points for p in cl)
        ]
        final_complete_clusters.extend(refinement_clusters)
        
        # Ricalcola performance finali
        final_performance = pc.calc_clusters_stats_ON(
            final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        if verbose:
            print(f"\n✅ REFINEMENT COMPLETATO")
            print(f"   • Nuovi cluster totali: {len(final_complete_clusters)}")
            excessive_after = count_clusters_with_excessive_overtime(final_performance, threshold=1)
            print(f"   • Cluster con overtime > 1: {excessive_after} (prima: {excessive_overtime_count})")
        
        return all_results, final_complete_clusters, final_performance, True
    
    return all_results, final_complete_clusters, final_performance, False





def plot_iteration_results(all_results, delivery_points, iteration_to_plot=None):
    """
    Plotta i risultati di una specifica iterazione o dell'ultima
    """
    if iteration_to_plot is None:
        iteration_to_plot = max(all_results.keys())
    
    if iteration_to_plot not in all_results:
        print(f"Iterazione {iteration_to_plot} non trovata")
        return
    
    results = all_results[iteration_to_plot]
    clusters = results['clusters']
    threshold = results['threshold']
    accepted = results['accepted_clusters']
    
    plot_clusters(clusters, delivery_points, 
                  f"Iterazione {iteration_to_plot} - Soglia {threshold} min "
                  f"({len(accepted)} accettati/{len(clusters)} totali)")



# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# SOLO PER DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


def diagnose_point_removal(remaining_points_before, accepted_clusters, remaining_points_after, iteration):
    """Diagnostica dettagliata per capire la discrepanza"""
    
    print(f"\n🔍 DIAGNOSTICA ITERAZIONE {iteration}:")
    
    # 1. Conta punti prima
    points_before = len(remaining_points_before)
    print(f"   📊 Punti prima: {points_before}")
    
    # 2. Conta punti negli cluster accettati
    accepted_location_ids = set()
    total_accepted_points = 0
    for i, cluster in enumerate(accepted_clusters):
        cluster_size = len(cluster)
        total_accepted_points += cluster_size
        accepted_location_ids.update(cluster)
        print(f"   ✅ Cluster {i+1}: {cluster_size} punti")
    
    print(f"   📊 Totale punti accettati: {total_accepted_points}")
    print(f"   📊 Location_ID unici accettati: {len(accepted_location_ids)}")
    
    # 3. Verifica che tutti i location_id accettati esistano in remaining_points_before
    existing_ids = set(remaining_points_before['location_id'])
    missing_ids = accepted_location_ids - existing_ids
    if missing_ids:
        print(f"   ⚠️ PROBLEMA: {len(missing_ids)} ID accettati non esistono in remaining_points!")
        print(f"      ID mancanti: {list(missing_ids)[:10]}...")
    
    # 4. Simula la rimozione manualmente
    should_remain = remaining_points_before[
        ~remaining_points_before['location_id'].isin(accepted_location_ids)
    ]
    expected_remaining = len(should_remain)
    
    # 5. Conta punti dopo
    points_after = len(remaining_points_after)
    print(f"   📊 Punti dopo rimozione: {points_after}")
    print(f"   📊 Punti attesi dopo rimozione: {expected_remaining}")
    
    # 6. Calcola discrepanze
    actual_removed = points_before - points_after
    expected_removed = len(accepted_location_ids)
    discrepancy = total_accepted_points - actual_removed
    
    print(f"   📊 Rimossi effettivamente: {actual_removed}")
    print(f"   📊 Dovevano essere rimossi: {expected_removed}")
    print(f"   📊 DISCREPANZA: {discrepancy}")
    
    if discrepancy != 0:
        print(f"   ❌ DISCREPANZA RILEVATA!")
        
        # Analisi più profonda
        if points_after != expected_remaining:
            print(f"   🔍 Il DataFrame finale non corrisponde alla simulazione")
            print(f"   🔍 Differenza: {points_after - expected_remaining}")
    
    print(f"   ✅ Diagnostica completata\n")
    
    return {
        'points_before': points_before,
        'total_accepted_points': total_accepted_points,
        'actual_removed': actual_removed,
        'discrepancy': discrepancy
    }

# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# FINE DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


In [6]:
import time
start = time.time()

all_results, final_clusters, final_performance, refinement_applied = heuristic_iterative_clustering_complete(
    delivery_points=pc.delivery_points_ON,
    depot_location=pc.depot_location,
    initial_threshold=800,
    increment_threshold=100,
    unload_min=10,
    max_iterations=15,
    verbose=True,
    enable_diagnostics=True  # Metti False per disabilitare la diagnostica
)


end = time.time()
print(f"Tempo di esecuzione algoritmo: {(end - start)/60:.2f} min")

# Stampa risultato refinement
if refinement_applied:
    print("\n🎯 REFINEMENT APPLICATO con successo!")
else:
    print("\n✅ NESSUN REFINEMENT necessario - soluzione ottimale trovata")

🔄 INIZIO clustering iterativo COMPLETO con soglia crescente
   • Soglia iniziale: 800 min
   • Incremento: +100 min per iterazione

🔄 Iterazione 1: soglia = 800 min
   • Punti rimanenti: 3219
      ✅ Cluster 1: 40 punti, max_mean=395.2min, overtime=0, motivo=mean_range
      ❌ Cluster 10: 53 punti, max_mean=277.6min, overtime=0, RIFIUTATO
      ❌ Cluster 11: 54 punti, max_mean=305.8min, overtime=0, RIFIUTATO
      ❌ Cluster 12: 57 punti, max_mean=300.9min, overtime=0, RIFIUTATO
      ❌ Cluster 13: 55 punti, max_mean=222.0min, overtime=0, RIFIUTATO
      ❌ Cluster 14: 54 punti, max_mean=236.3min, overtime=0, RIFIUTATO
      ❌ Cluster 15: 50 punti, max_mean=232.0min, overtime=0, RIFIUTATO
      ❌ Cluster 16: 53 punti, max_mean=313.3min, overtime=0, RIFIUTATO
      ❌ Cluster 17: 56 punti, max_mean=236.6min, overtime=0, RIFIUTATO
      ❌ Cluster 18: 54 punti, max_mean=292.9min, overtime=0, RIFIUTATO
      ❌ Cluster 19: 59 punti, max_mean=338.3min, overtime=0, RIFIUTATO
      ❌ Cluster 2: 4

### salvataggio risultati

In [ ]:

final_performance.to_csv('clustering_methods_performances/euristic_complete_v2_ON_5.csv')

with open('cluster_dicts/cluster_dict_euristic_complete_v2_ON_5.pkl', 'wb') as f:
    pickle.dump(final_clusters, f)

# 3 full

In [8]:

def clustering_euristico_multicluster(full_time_mat_min: np.ndarray,
                                      index_to_location_id: list[int],
                                      delivery_points: pd.DataFrame,
                                      max_time: int = 500,
                                      unloading_time: int = 10):
    """
    Algoritmo Cluster‐First, Route‐Second:
    • seed = punto più distante dal deposito
    • greedy: aggiunge il punto che allunga meno il tour
    • stop quando supererebbe max_time
    """
    n_points = full_time_mat_min.shape[0]
    unassigned = set(range(1, n_points))          # deposito è indice 0
    clusters = []


    while unassigned:
        # --- nuovo cluster
        first_idx = max(unassigned, key=lambda i: full_time_mat_min[0, i])
        cluster = [first_idx]
        unassigned.remove(first_idx)
        cluster_time = full_time_mat_min[0, first_idx] * 2 + unloading_time


        # --- inserimento greedy
        while unassigned:
            # tempo minimo dal cluster al candidato
            cand, min_cost = min(
                ((c, min(full_time_mat_min[p, c] for p in cluster)) for c in unassigned),
                key=lambda x: x[1]
            )
            if cluster_time + min_cost + unloading_time > max_time:
                break
            cluster.append(cand)
            unassigned.remove(cand)
            cluster_time += min_cost + unloading_time


        clusters.append([index_to_location_id[i] for i in cluster])


    return clusters



def plot_clusters(clusters, delivery_points, title):
    """Visual plot rapido"""
    cmap = plt.cm.get_cmap("tab20", len(clusters))
    plt.figure(figsize=(10, 7))
    for i, cl in enumerate(clusters):
        sub = delivery_points[delivery_points["location_id"].isin(cl)]
        plt.scatter(sub["lon"], sub["lat"], c=[cmap(i)],
                    label=f"C{i+1}", alpha=.7, s=20, edgecolors="k")
    plt.title(title)
    plt.xlabel("lon"); plt.ylabel("lat")
    plt.legend(ncol=2, fontsize="small")
    plt.show()


def deduplicate_clusters(clusters, already_assigned_points):
    """
    Rimuove punti già assegnati dai cluster e restituisce solo cluster validi
    
    Args:
        clusters: lista di cluster (liste di location_id)
        already_assigned_points: set di location_id già assegnati
        
    Returns:
        result: lista di cluster puliti
        assigned: set aggiornato di punti assegnati
    """
    assigned = set(already_assigned_points)
    result = []
    
    for cluster in clusters:
        clean_cluster = [loc_id for loc_id in cluster if loc_id not in assigned]
        if clean_cluster:  # Solo se il cluster ha almeno un punto
            result.append(clean_cluster)
            assigned.update(clean_cluster)
    
    return result, assigned


def get_last_iteration_rejected_clusters(all_results, already_assigned_points, verbose=True):
    """
    Recupera i cluster RIFIUTATI dell'ULTIMA iterazione, filtrando i punti già assegnati
    
    Args:
        all_results: dizionario con i risultati di tutte le iterazioni
        already_assigned_points: set di location_id già assegnati
        verbose: stampa messaggi di debug
        
    Returns:
        lista di cluster filtrati
    """
    if not all_results:
        return []
    
    # Prendi l'ultima iterazione
    last_iteration = max(all_results.keys())
    last_results = all_results[last_iteration]
    
    # Recupera i cluster rifiutati
    rejected_cluster_indices = last_results['rejected_indices']
    all_clusters = last_results['clusters']
    rejected_clusters = [all_clusters[i] for i in rejected_cluster_indices]
    
    if verbose:
        print(f"   🔍 Recuperando cluster rifiutati dall'ultima iterazione ({last_iteration})")
        print(f"   📋 Cluster rifiutati disponibili: {len(rejected_clusters)}")
    
    # Filtra punti già assegnati
    filtered_clusters, _ = deduplicate_clusters(rejected_clusters, already_assigned_points)
    
    if verbose:
        total_points = sum(len(cl) for cl in filtered_clusters)
        print(f"   ✅ Cluster recuperati dopo filtro: {len(filtered_clusters)} ({total_points} punti)")
    
    return filtered_clusters


def count_clusters_with_excessive_overtime(performance_df, threshold=1):
    """
    Conta quanti cluster hanno più di 'threshold' giorni di overtime
    
    Args:
        performance_df: DataFrame con le performance dei cluster
        threshold: soglia di overtime (default 1)
        
    Returns:
        numero di cluster con overtime > threshold
    """
    count = 0
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        total_overtime = cluster_data['n_overtime_days'].sum()
        if total_overtime > threshold:
            count += 1
    return count



def evaluate_and_accept_clusters(clusters, performance_df, verbose=True):
    """
    Valuta i cluster e decide quali accettare basandosi sui criteri:
    1) Media giornaliera tra 390-480 minuti
    2) Overtime accettabile (massimo 1 giorno per tipo di giorno della settimana)
    
    Returns:
        accepted_clusters: list di cluster (liste di location_id) accettati
        rejected_indices: list di indici dei cluster rifiutati
    """
    accepted_clusters = []
    rejected_indices = []
    
    # Raggruppa performance per cluster
    for cluster_name, cluster_data in performance_df.groupby('cluster'):
        cluster_index = int(cluster_name.split()[-1]) - 1  # "Cluster 1" -> index 0
        cluster = clusters[cluster_index]
        
        # CRITERIO 1: Media giornaliera tra 390-480 minuti
        mean_minutes_range = cluster_data['mean_minutes'].between(390, 480).any()
        
        # CRITERIO 2: Overtime accettabile
        overtime_acceptable = check_overtime_acceptable(cluster_data)
        
        if mean_minutes_range or overtime_acceptable:
            accepted_clusters.append(cluster)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                reason = "mean_range" if mean_minutes_range else "overtime_ok"
                print(f"      ✅ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"motivo={reason}")
        else:
            rejected_indices.append(cluster_index)
            if verbose:
                max_mean = cluster_data['mean_minutes'].max()
                total_overtime = cluster_data['n_overtime_days'].sum()
                print(f"      ❌ {cluster_name}: {len(cluster)} punti, "
                      f"max_mean={max_mean:.1f}min, overtime={total_overtime}, "
                      f"RIFIUTATO")
    
    return accepted_clusters, rejected_indices



def check_overtime_acceptable(cluster_data):
    """
    Controlla se l'overtime è accettabile:
    massimo 1 giorno di overtime per ogni giorno della settimana
    
    Args:
        cluster_data: DataFrame con performance di un singolo cluster
    
    Returns:
        bool: True se overtime accettabile, False altrimenti
    """
    # Controlla per ogni giorno della settimana
    for _, row in cluster_data.iterrows():
        if row['n_overtime_days'] > 1:  # Più di 1 giorno di overtime per questo weekday -> non accettato
            return False
        elif row['n_overtime_days'] == 1:
            return True  # Ok, 1 giorno di overtime è accettabile
    
    return False






def assemble_final_performance_from_cache(all_results, final_complete_clusters, verbose=True):
    """
    Assembla un DataFrame finale dalle performance già calcolate nelle iterazioni.
    Molto più veloce del ricalcolo completo.
    """
    if verbose:
        print("📋 Assemblando performance finali da cache esistenti...")
    
    final_performance_pieces = []
    cluster_counter = 1
    
    for cluster in final_complete_clusters:
        cluster_found = False
        
        # Cerca questo cluster nelle iterazioni passate
        for iteration_num, results in all_results.items():
            for i, iter_cluster in enumerate(results['clusters']):
                if set(iter_cluster) == set(cluster):  # Stesso cluster
                    # Prendi le performance già calcolate
                    iter_perf_df = results['performance_df']
                    cluster_name_old = f"Cluster {i+1}"
                    cluster_data = iter_perf_df[iter_perf_df['cluster'] == cluster_name_old].copy()
                    
                    # Rinomina con il nuovo indice
                    cluster_data['cluster'] = f"Cluster {cluster_counter}"
                    final_performance_pieces.append(cluster_data)
                    cluster_counter += 1
                    cluster_found = True
                    break
            
            if cluster_found:
                break
        
        if not cluster_found and verbose:
            print(f"   ⚠️ Cluster non trovato in cache - dovrà essere ricalcolato")
    
    if final_performance_pieces:
        final_df = pd.concat(final_performance_pieces, ignore_index=True)
        if verbose:
            print(f"   ✅ Assemblate performance per {len(final_performance_pieces)} cluster da cache")
        return final_df
    else:
        if verbose:
            print("   ⚠️ Nessuna performance in cache - calcolo completo necessario")
        return None



def heuristic_iterative_clustering_complete(delivery_points: pd.DataFrame,
                                           depot_location: tuple[float, float],
                                           initial_threshold: int = 800,
                                           increment_threshold: int = 120,
                                           unload_min: int = 10,
                                           max_iterations: int = 10,
                                           verbose: bool = True,
                                           enable_diagnostics: bool = False):
    """
    Algoritmo iterativo che restituisce TUTTI i punti assegnati a cluster.
    MODIFICHE:
    - Ogni punto assegnato ad un solo cluster (no duplicati)
    - Fallback all'ultima iterazione per punti non assegnati
    - Refinement automatico se >5 cluster con overtime >1
    
    Returns:
        all_results: dict con tutti i risultati intermedi
        final_complete_clusters: list di TUTTI i cluster (accettati + migliori fallback)
        final_performance: DataFrame con performance di tutti i cluster finali
        refinement_applied: bool indicante se è stato applicato refinement
    """
    
    if verbose:
        print("🔄 INIZIO clustering iterativo COMPLETO con soglia crescente")
        print(f"   • Soglia iniziale: {initial_threshold} min")
        print(f"   • Incremento: +{increment_threshold} min per iterazione")
    
    all_results = {}
    remaining_points = delivery_points.copy()
    original_points = set(delivery_points['location_id'])
    already_assigned_points = set()  # NUOVO: Traccia globale dei punti già assegnati
    current_threshold = initial_threshold
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n🔄 Iterazione {iteration + 1}: soglia = {current_threshold} min")
            print(f"   • Punti rimanenti: {len(remaining_points)}")
        
        if remaining_points.empty:
            if verbose:
                print("   ✅ Nessun punto rimanente - STOP")
            break
        
        # ================== CLUSTERING ==================
        dm, tm, idx2loc, _ = pc.distance_matrix_creation('', remaining_points)
        clusters = clustering_euristico_multicluster(
            tm, idx2loc, remaining_points,
            max_time=current_threshold,
            unloading_time=unload_min
        )
        
        if not clusters:
            if verbose:
                print("   ⚠️ Nessun cluster generato - STOP")
            break
        
        # ================== PERFORMANCE ==================
        performance_df = pc.calc_clusters_stats(
            clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        # ================== VALUTAZIONE E ACCETTAZIONE ==================
        accepted_clusters, rejected_cluster_indices = evaluate_and_accept_clusters(
            clusters, performance_df, verbose=verbose
        )
        
        # Salva risultati dell'iterazione (con rejected_indices invece di rejected_clusters)
        all_results[iteration + 1] = {
            'threshold': current_threshold,
            'clusters': clusters,
            'performance_df': performance_df,
            'accepted_clusters': accepted_clusters,
            'rejected_indices': rejected_cluster_indices
        }
        
        if verbose:
            print(f"   ✅ Cluster accettati: {len(accepted_clusters)}/{len(clusters)}")
        
        # ================== RIMOZIONE PUNTI ACCETTATI (NO DUPLICATI) ==================
        accepted_location_ids = set()
        for cluster in accepted_clusters:
            accepted_location_ids.update(cluster)
        
        # NUOVO: Filtra solo gli ID che esistono effettivamente nel DataFrame
        existing_ids = set(remaining_points['location_id'])
        valid_accepted_ids = accepted_location_ids.intersection(existing_ids)
        missing_ids = accepted_location_ids - existing_ids
        
        if verbose and missing_ids:
            print(f'   ⚠️ ID nei cluster ma non nel DataFrame: {len(missing_ids)} ID')
            print(f'   🔧 ID validi da rimuovere: {len(valid_accepted_ids)} ID')
        
        # NUOVO: Aggiorna il set globale dei punti assegnati
        already_assigned_points.update(valid_accepted_ids)

        # PER LA DIAGNOSTICA (opzionale):
        if enable_diagnostics and verbose:
            remaining_points_after_removal = remaining_points[
                ~remaining_points["location_id"].isin(valid_accepted_ids)
            ].copy()
            diagnose_point_removal(remaining_points, accepted_clusters, 
                remaining_points_after_removal, iteration + 1)
        
        # USA VALID_ACCEPTED_IDS invece di ACCEPTED_LOCATION_IDS
        remaining_points = remaining_points[
            ~remaining_points['location_id'].isin(valid_accepted_ids)
        ].copy()
        
        # Se tutti i cluster sono stati accettati, abbiamo finito
        if len(accepted_clusters) == len(clusters):
            if verbose:
                print("   🎯 Tutti i cluster sono stati accettati - STOP")
            break
        
        # Incrementa soglia per prossima iterazione
        current_threshold += increment_threshold
    
    # ================== ASSEMBLAGGIO FINALE ==================
    if verbose:
        print(f"\n🔧 Assemblaggio soluzione finale...")
    
    final_complete_clusters = []
    
    # 1. Aggiungi tutti i cluster accettati (già senza duplicati)
    for iteration_results in all_results.values():
        for cluster in iteration_results['accepted_clusters']:
            final_complete_clusters.append(cluster)
    
    # 2. NUOVO: Per i punti rimanenti, usa i cluster RIFIUTATI dell'ULTIMA ITERAZIONE
    unassigned_points = original_points - already_assigned_points
    
    if unassigned_points:
        if verbose:
            print(f"   📋 Punti non assegnati: {len(unassigned_points)}")
            print("   🔍 Recuperando cluster rifiutati dall'ultima iterazione...")
        
        last_iteration_clusters = get_last_iteration_rejected_clusters(
            all_results, already_assigned_points, verbose
        )
        
        final_complete_clusters.extend(last_iteration_clusters)
        for cluster in last_iteration_clusters:
            already_assigned_points.update(cluster)
    
    # ================== PERFORMANCE FINALI ==================
    if verbose:
        print(f"\n📊 Calcolando performance finali per {len(final_complete_clusters)} cluster...")
    
    # Calcola performance (senza cache per ora, per evitare problemi)
    final_performance = pc.calc_clusters_stats(
        final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
    )
    
    if verbose:
        total_points = sum(len(cluster) for cluster in final_complete_clusters)
        print(f"\n🏁 COMPLETATO:")
        print(f"   • Iterazioni eseguite: {len(all_results)}")
        print(f"   • Cluster finali totali: {len(final_complete_clusters)}")
        print(f"   • Punti totali assegnati: {total_points}")
        print(f"   • Copertura: {total_points}/{len(original_points)} punti")
        
        missing = len(original_points) - total_points
        if missing > 0:
            print(f"   ⚠️ Punti mancanti: {missing}")
    
    # ================== CHECK OVERTIME ECCESSIVO E REFINEMENT ==================
    excessive_overtime_count = count_clusters_with_excessive_overtime(final_performance, threshold=1)
    
    if verbose:
        print(f"\n📊 Cluster con più di 1 giorno overtime: {excessive_overtime_count}")
    
    # NUOVO: Se ci sono più di 5 cluster con overtime eccessivo, applica refinement
    refinement_needed = excessive_overtime_count > 5
    
    if refinement_needed:
        if verbose:
            print(f"\n🔄 REFINEMENT NECESSARIO: {excessive_overtime_count} cluster con overtime > 1")
            print("   🚀 Avvio 5 cicli aggiuntivi con soglia 1550 min (+60/iterazione)...")
        
        # Identifica i punti nei cluster problematici
        problematic_points = set()
        problematic_cluster_names = []
        for cluster_name, cluster_data in final_performance.groupby('cluster'):
            total_overtime = cluster_data['n_overtime_days'].sum()
            if total_overtime > 1:
                problematic_cluster_names.append(cluster_name)
                cluster_index = int(cluster_name.split()[-1]) - 1
                problematic_cluster = final_complete_clusters[cluster_index]
                problematic_points.update(problematic_cluster)
        
        # Crea DataFrame con i punti problematici
        refinement_points = delivery_points[
            delivery_points['location_id'].isin(problematic_points)
        ].copy()
        
        if verbose:
            print(f"   📋 Cluster problematici: {len(problematic_cluster_names)}")
            print(f"   📋 Punti da riclusterare: {len(refinement_points)}")
        
        # Applica refinement (ricorsivo)
        refinement_results, refinement_clusters, refinement_performance, _ = heuristic_iterative_clustering_complete(
            delivery_points=refinement_points,
            depot_location=depot_location,
            initial_threshold=1550,  # NUOVO: soglia più alta
            increment_threshold=60,   # NUOVO: incremento più piccolo
            unload_min=unload_min,
            max_iterations=5,        # NUOVO: max 5 iterazioni
            verbose=verbose,
            enable_diagnostics=False  # Disabilita diagnostica nel refinement
        )
        
        # Sostituisci i cluster problematici con quelli raffinati
        final_complete_clusters = [
            cl for cl in final_complete_clusters 
            if not any(p in problematic_points for p in cl)
        ]
        final_complete_clusters.extend(refinement_clusters)
        
        # Ricalcola performance finali
        final_performance = pc.calc_clusters_stats(
            final_complete_clusters, time_limit=3, parallel=True, max_workers=7, verbose=False
        )
        
        if verbose:
            print(f"\n✅ REFINEMENT COMPLETATO")
            print(f"   • Nuovi cluster totali: {len(final_complete_clusters)}")
            excessive_after = count_clusters_with_excessive_overtime(final_performance, threshold=1)
            print(f"   • Cluster con overtime > 1: {excessive_after} (prima: {excessive_overtime_count})")
        
        return all_results, final_complete_clusters, final_performance, True
    
    return all_results, final_complete_clusters, final_performance, False





def plot_iteration_results(all_results, delivery_points, iteration_to_plot=None):
    """
    Plotta i risultati di una specifica iterazione o dell'ultima
    """
    if iteration_to_plot is None:
        iteration_to_plot = max(all_results.keys())
    
    if iteration_to_plot not in all_results:
        print(f"Iterazione {iteration_to_plot} non trovata")
        return
    
    results = all_results[iteration_to_plot]
    clusters = results['clusters']
    threshold = results['threshold']
    accepted = results['accepted_clusters']
    
    plot_clusters(clusters, delivery_points, 
                  f"Iterazione {iteration_to_plot} - Soglia {threshold} min "
                  f"({len(accepted)} accettati/{len(clusters)} totali)")



# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# SOLO PER DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


def diagnose_point_removal(remaining_points_before, accepted_clusters, remaining_points_after, iteration):
    """Diagnostica dettagliata per capire la discrepanza"""
    
    print(f"\n🔍 DIAGNOSTICA ITERAZIONE {iteration}:")
    
    # 1. Conta punti prima
    points_before = len(remaining_points_before)
    print(f"   📊 Punti prima: {points_before}")
    
    # 2. Conta punti negli cluster accettati
    accepted_location_ids = set()
    total_accepted_points = 0
    for i, cluster in enumerate(accepted_clusters):
        cluster_size = len(cluster)
        total_accepted_points += cluster_size
        accepted_location_ids.update(cluster)
        print(f"   ✅ Cluster {i+1}: {cluster_size} punti")
    
    print(f"   📊 Totale punti accettati: {total_accepted_points}")
    print(f"   📊 Location_ID unici accettati: {len(accepted_location_ids)}")
    
    # 3. Verifica che tutti i location_id accettati esistano in remaining_points_before
    existing_ids = set(remaining_points_before['location_id'])
    missing_ids = accepted_location_ids - existing_ids
    if missing_ids:
        print(f"   ⚠️ PROBLEMA: {len(missing_ids)} ID accettati non esistono in remaining_points!")
        print(f"      ID mancanti: {list(missing_ids)[:10]}...")
    
    # 4. Simula la rimozione manualmente
    should_remain = remaining_points_before[
        ~remaining_points_before['location_id'].isin(accepted_location_ids)
    ]
    expected_remaining = len(should_remain)
    
    # 5. Conta punti dopo
    points_after = len(remaining_points_after)
    print(f"   📊 Punti dopo rimozione: {points_after}")
    print(f"   📊 Punti attesi dopo rimozione: {expected_remaining}")
    
    # 6. Calcola discrepanze
    actual_removed = points_before - points_after
    expected_removed = len(accepted_location_ids)
    discrepancy = total_accepted_points - actual_removed
    
    print(f"   📊 Rimossi effettivamente: {actual_removed}")
    print(f"   📊 Dovevano essere rimossi: {expected_removed}")
    print(f"   📊 DISCREPANZA: {discrepancy}")
    
    if discrepancy != 0:
        print(f"   ❌ DISCREPANZA RILEVATA!")
        
        # Analisi più profonda
        if points_after != expected_remaining:
            print(f"   🔍 Il DataFrame finale non corrisponde alla simulazione")
            print(f"   🔍 Differenza: {points_after - expected_remaining}")
    
    print(f"   ✅ Diagnostica completata\n")
    
    return {
        'points_before': points_before,
        'total_accepted_points': total_accepted_points,
        'actual_removed': actual_removed,
        'discrepancy': discrepancy
    }

# ------------------------------------------------
# ----------------------------------------------------------------------------------------
# FINE DEBUG
# ----------------------------------------------------------------------------------------
# ------------------------------------------------


In [ ]:

import time
start = time.time()

all_results, final_clusters, final_performance, refinement_applied = heuristic_iterative_clustering_complete(
    delivery_points=pc.delivery_points,
    depot_location=pc.depot_location,
    initial_threshold=800,
    increment_threshold=100,
    unload_min=10,
    max_iterations=15,
    verbose=True,
    enable_diagnostics=True  # Metti False per disabilitare la diagnostica
)


end = time.time()
print(f"Tempo di esecuzione algoritmo: {(end - start)/60:.2f} min")

# Stampa risultato refinement
if refinement_applied:
    print("\n🎯 REFINEMENT APPLICATO con successo!")
else:
    print("\n✅ NESSUN REFINEMENT necessario - soluzione ottimale trovata")

🔄 INIZIO clustering iterativo COMPLETO con soglia crescente
   • Soglia iniziale: 800 min
   • Incremento: +100 min per iterazione

🔄 Iterazione 1: soglia = 800 min
   • Punti rimanenti: 3764
      ✅ Cluster 1: 42 punti, max_mean=413.9min, overtime=2, motivo=mean_range
      ❌ Cluster 10: 51 punti, max_mean=218.2min, overtime=0, RIFIUTATO
      ❌ Cluster 11: 53 punti, max_mean=250.2min, overtime=0, RIFIUTATO
      ❌ Cluster 12: 52 punti, max_mean=201.7min, overtime=0, RIFIUTATO
      ❌ Cluster 13: 50 punti, max_mean=296.5min, overtime=0, RIFIUTATO
      ❌ Cluster 14: 55 punti, max_mean=236.1min, overtime=0, RIFIUTATO
      ❌ Cluster 15: 55 punti, max_mean=205.8min, overtime=0, RIFIUTATO
      ❌ Cluster 16: 50 punti, max_mean=216.6min, overtime=0, RIFIUTATO
      ❌ Cluster 17: 50 punti, max_mean=228.4min, overtime=0, RIFIUTATO
      ❌ Cluster 18: 56 punti, max_mean=219.2min, overtime=0, RIFIUTATO
      ❌ Cluster 19: 56 punti, max_mean=210.7min, overtime=0, RIFIUTATO
      ❌ Cluster 2: 4

### salvataggio risultati

In [ ]:

final_performance.to_csv('clustering_methods_performances/euristic_complete_v2_4.csv')

with open('cluster_dicts/cluster_dict_euristic_complete_v2_4.pkl', 'wb') as f:
    pickle.dump(final_clusters, f)